In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram

# Chapitre 4: Systèmes à plusieurs qubits

A présent, étudions un système à deux qubits. Il s'agit du système minimal pour étudier l'un des plus fascinants phénomènes prédit par la mécanique quantique: l'intrication quantique. Celle-ci est à la base de l'accélération dans les calculs de certains problèmes et est cruciale en information et en cryptographie quantiques.

Afin d'étudier des systèmes à plusieurs qubits, introduisons tout d'abord les notations utiles.

## 4.1: Représentation tensorielle des états quantiques et intrication

Comme nous l'avons vu dans les chapitres précédents, l'état d'un qubit est représenté par un ket $\ket \psi_1^{} = \alpha_1^{} \ket 0 + \beta_1^{} \ket{1}$. De même, l'état d'un second qubit se notera $\ket \phi_2^{} = \alpha_2^{} \ket 0 + \beta_2^{} \ket{1}$. L'état le plus général d'un système composé de deux qubits se note quant à lui

$\qquad \ket{\Psi_2} = a_{00}^{} \ket 0_2^{} \otimes \ket 0_1^{} + a_{01}^{} \ket 0_2^{} \otimes \ket 1_1^{} + a_{10}^{} \ket 1_2^{} \otimes \ket 0_1^{} + a_{11}^{} \ket 1_2^{} \otimes \ket 1_1^{}$,

où le symbole $\otimes$ est appelé *produit tensoriel*. On remarque l'ajout des indices "1" et "2" aux kets $\ket 0$ et $\ket 1$ afin de notifier à quel qubit ils correspondent. Bien que claire, la notation tensorielle et avec les indices peut s'avérer lourde. Lorsqu'il est aisé de retracer quelle composante appartient à quel qubit, on peut s'en passer et même encore plus compacter les notations en fusionnat les kets: $\ket{0}_2^{} \otimes \ket{0}_1^{} = \ket{00}$. Ainsi,

$\qquad \ket{\Psi_2} = a_{00}^{} \ket{00} + a_{01}^{} \ket{01} + a_{10}^{} \ket{10} + a_{11}^{} \ket{11}$. 

Plus tard, nous utiliserons cette notation mais pour le moment, gardons la notation explicite. 

Par ailleurs, on comprend que le nombre d'états de base s'élève à 4, soit deux par qubit. Ainsi, avec 3 qubits, le nombre d'états de base s'élève à 8, et pour $N$ qubits, à $2^N_{}$. Dès lors, un état à $2$ qubits s'écrit avec des vecteurs à $2^2_{}$ composantes. Pour deux qubits, on a donc

$\begin{pmatrix} 
a_{00}^{} \\
a_{01}^{} \\
a_{10}^{}\\
a_{11}^{} \\
\end{pmatrix}.$

<!-- $\begin{pmatrix} 
\alpha_2^{} \alpha_1^{} \\
\alpha_2^{} \beta_1^{} \\
\beta_2^{} \alpha_1^{}\\
\beta_2^{} \beta_1^{} \\
\end{pmatrix}$ -->

### Etats produits (ou séparables)

Certains états (mais pas tous, et c'est important!) d'un système à deux qubits peuvent s'écrire en *séparant* les états du premier et du second qubits: 

$\qquad \Psi_2^{} =  \ket \phi_2^{} \otimes \ket \psi_1^{} = \left(\alpha_2^{} \ket 0 + \beta_2^{} \ket{1}\right) \otimes \left(\alpha_1^{} \ket 0 + \beta_1^{} \ket{1}\right)$

$\ \ \quad \qquad = \alpha_2^{} \alpha_1^{} \ket 0 \otimes \ket 0 + \alpha_2^{} \beta_1^{} \ket 0 \otimes \ket 1 + \beta_2^{} \alpha_1^{} \ket 1 \otimes \ket 0 + \beta_2^{} \beta_1^{} \ket 1 \otimes \ket 1$

$\ \ \quad \qquad = \alpha_2^{} \alpha_1^{} \ket{00} + \alpha_2^{} \beta_1^{} \ket 0 \otimes \ket 1 + \beta_2^{} \alpha_1^{} \ket 1 \otimes \ket 0 + \beta_2^{} \beta_1^{} \ket 1 \otimes \ket 1$

On parle alors d'**états séparables**. Ceci se produit lorsque les qubits évoluent indépendamment l'un de l'autre, c'est-à-dire qu'intéragir avec un qubit n'a aucune influence sur l'autre qubit.

### Etats mixtes et intrication

Les autres états à deux qubits ne sont pas séparables, on ne peut donc pas les écrire comme un produit des états de chaque qubit. Ceci veut aussi dire qu'intéragir ou mesurer l'état d'un qubit a une influence sur l'état de l'autre qubit. C'est ici qu'entre en jeu l'**intrication quantique**. Des exemples importants d'états intriqués sont les **états de Bell**. Nous verrons plus loin comment les créer à partir de deux qubits initialement indépendents.

Paires de Bell:

$\qquad \ket{\Phi_+^{}} = \frac{1}{\sqrt{2}} (\ket{00} + \ket{11})$,

$\qquad \ket{\Phi_-^{}} = \frac{1}{\sqrt{2}} (\ket{00} - \ket{11})$,

$\qquad \ket{\Psi_+^{}} = \frac{1}{\sqrt{2}} (\ket{01} + \ket{10})$,

$\qquad \ket{\Psi_-^{}} = \frac{1}{\sqrt{2}} (\ket{01} - \ket{10})$.

Comme nous le disions précédemment, ces états ne sont pas séparables! Un phénomène singulier peut être observé grâce à une paire des qubits intriqués. Disons que l'on ait pu produire la paire de Bell $\ket{\Phi_+^{}}$. Notons que chaque qubit peut se trouver dans une superposition d'états $\ket 0$ et $\ket 1$, mais avec une condition, imposée par l'état de Bell $\ket{\Phi_+^{}}$ avant leur séparation: mesurer l'un dans l'état 0 implique que l'autre est aussi dans l'état $\ket 0$ (et idem pour l'état $\ket 1$).

En résumé, les qubits sont chacun dans un état superposé mais le résulat d'une mesure de l'un déterminera nécessairement l'état de l'autre qubit, qui, avant la mesure, était aussi dans un état superposé! Cette correlation due au conditionnement de la mesure est appelée **intrication quantique**.

$~$

## 4.2: Opérations sur plusieurs qubits

Des portes quantiques peuvent s'appliquer séparément ou simultanément sur plusieurs qubits à la fois.

### Portes à un qubit

Il est possible d'appliquer des portes logiques séparément sur des qubits. En se souvenant que l'action d'une porte logique (à un seul  qubit) sur un qubit correspond à une multiplication par une matrice (unitaire) sur le vecteur représentatn le qubit, il suffit d'appliquer une matrice sur le premier qubit et séparément une deuxième matrice sur le deuxième qubit:

$\qquad (M_2^{} \ket \psi_2^{}) \otimes (M_1^{} \ket \phi_1^{}) =  M_2^{} \otimes M_1^{} (\ket \psi_2^{} \otimes \ket \phi_1^{})$.

Ceci est montré sur le circuit suivant, où deux portes de Hadamard appliquées chacune sur un qubit:

In [ ]:
# 1: Créer un objet "circuit à 1 qubit":
qc = QuantumCircuit(2)

# 2: Definir l'état initial des deux qubits:
etat_initial_1 = [1, 0]  # l'état |0>
qc.initialize(etat_initial_1, 0)
etat_initial_2 = [0, 1]  # l'état |0>
qc.initialize(etat_initial_2, 1)

# 3: Ajouter la porte de Hadamard sur chaque qubit:
qc.h(0)
qc.h(1)

# 4: Appliquer le circuit sur l'état initial:
etat_final = Statevector.from_instruction(qc)

print("Etat du système à l'issu du circuit:", etat_final)

# Schéma du circuit
qc.draw('mpl')

Après que les qubits dans les états $\ket 0$ et $\ket 1$ soient passés chacun de leur côté par une porte de Hadamard, l'état final de l'ensemble des deux qubits s'écrit:

$\qquad (H \ket 1) \otimes (H\ket{0}) = (\frac{1}{\sqrt{2}} \ket 0 - \frac{1}{\sqrt{2}} \ket 1) \otimes (\frac{1}{\sqrt{2}} \ket 0 + \frac{1}{\sqrt{2}} \ket 1)$ 

$\qquad \qquad \qquad \qquad \quad \ \ = \frac{1}{2} \ket{00} + \frac{1}{2} \ket{01} - \frac{1}{2} \ket{10} - \frac{1}{2} \ket{11} $.

### Portes à 2 qubits

Il existe également des portes quantiques qui agissent nécessairement sur deux qubits à la fois (les matrices correspondantes ne peuvent alors pas s'écrire comme un seul produit tensoriel de deux matrices, mais bien comme une somme de produits tensoriels de deux matrices). Ce type de porte a pour effet d'*intriquer* les qubits. En voici quelques exemples.

La porte **CNOT** (ou **CX**) change l'état d'un qubit (de $\ket 0$ vers $\ket 1$ ou de $\ket 1$ vers $\ket 0$) si l'autre qubit est dans l'état $\ket 1$ et ne fait rien si ce dernier est dans l'état $\ket 0$. Ceci se traduit par l'action d'une matrice $4\times 4$ sur le vecteur représentant l'état de initial. Cette matrice s'écrit:

$C_X^{} = \begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & 1\\
0 & 0 & 1 & 0
\end{pmatrix}.$

En appliquant cette matrice sur l'état $\ket{10}$, représenté par $[0,0,1,0]$,

$\qquad C_X^{} \ket{10} = \begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & 1\\
0 & 0 & 1 & 0
\end{pmatrix} 
\begin{pmatrix} 0 \\ 0 \\ 1\\ 0
\end{pmatrix} = \begin{pmatrix} 0 \\ 0 \\ 0\\ 1
\end{pmatrix}  = \ket{11}.$


La porte **SWAP** échange les états des qubits. Elle est représentée par la matrice $4\times 4 suivante:

$S = \begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 0 & 1 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & 1
\end{pmatrix}$

Etudions son action sur l'état à deux qubits $\ket{01}$, représenté par le vecteur $[0,1,0,0]$:

$\qquad S\ket {01} = \begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 0 & 1 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & 1
\end{pmatrix} \begin{pmatrix} 0 \\ 1 \\ 0\\ 0
\end{pmatrix} = \begin{pmatrix} 0 \\ 0 \\ 1\\ 0
\end{pmatrix} = \ket{10}$, 

ce qui a bien échangé les états des qubits.

$~$

## 4.3: Etats de Bell

Décomp de Schmidt!!!!!!! Voir si entanglement ou non!

Voyons à présent comment créer une paire deux qubits intriqués et comment vérifier que les deux qubits sont bel et bien intriqués. Comme exemple, nous allons créer une *paire de Bell*, très utile en communication quantique. Pour créer l'état $\ket{\Phi_+^{}} = (\ket{00}+\ket{11})/\sqrt{2}$, on part de deux qubits dans l'état $\ket 0$. Sur l'un deux, on applique une porte de Hadamard, puis une porte CNOT. Ceci est réalisé grâce au code ci-dessous.

In [ ]:
# 1: Créer un objet "circuit à 1 qubit":
qc = QuantumCircuit(2)

# 2: Definir l'état initial:
psi_0 = [1, 0] # le qubit 0 est dans l'état |0>
psi_1 = [1, 0] # le qubit 1 est dans l'état |1>
qc.initialize(psi_0, 0)
qc.initialize(psi_1, 1)

# 3: Ajouter la porte de Hadamard au circuit:
qc.h(0)
qc.cx(0,1)

# 4: Appliquer le circuit sur l'état initial:
etat_final = Statevector(qc)

print("Etat final du système à l'issue du circuit:")
display(etat_final.draw('latex'))

# Schéma du circuit
qc.draw('mpl')

$~$

In [ ]:
## 4.3: Mesures et mesures partielles